In [1]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl


# 그래프 기본 테마 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 복잡한 통계 처리를 위한 라이브러리
from scipy import stats
from matplotlib.ticker import ScalarFormatter


In [2]:
# 데이터 불러오기
gj = pd.read_excel('data/거제벨버디어.xlsx')
npl = pd.read_excel('data/나폴리농원.xlsx')
dhs = pd.read_excel('data/대흥사.xlsx')
st = pd.read_excel('data/센텀스파랜드.xlsx')
scm = pd.read_excel('data/순천만국가정원.xlsx')
cdb = pd.read_excel('data/차덕분.xlsx')
forest = pd.read_excel('data/포레스트리솜.xlsx')
hb = pd.read_excel('data/한방진흥원.xlsx')
hbc = pd.read_excel('data/해비치.xlsx')

In [3]:
print(gj.columns)
print(npl.columns)
print(dhs.columns)
print(st.columns)   
print(scm.columns)
print(cdb.columns)
print(forest.columns)   
print(hb.columns)
print(hbc.columns)

rename = {
    '리뷰' : '리뷰',
    '작성자' : '작성자',
    '작성자정보' : '작성자정보',
    '날짜' : '날짜',
    '작성일' : '날짜',
    '방문회수' : '방문횟수',
    '방문횟수' : '방문횟수',
    '인증수단' : '인증수단',
    '업소명' : '업소명',
    '지역' : '지역',
}

gj.rename(columns=rename, inplace=True)
npl.rename(columns=rename, inplace=True)
dhs.rename(columns=rename, inplace=True)
st.rename(columns=rename, inplace=True)     
scm.rename(columns=rename, inplace=True)
cdb.rename(columns=rename, inplace=True)
forest.rename(columns=rename, inplace=True)
hb.rename(columns=rename, inplace=True)
hbc.rename(columns=rename, inplace=True)

print(gj.columns)
print(npl.columns)
print(dhs.columns)
print(st.columns)   
print(scm.columns)
print(cdb.columns)
print(forest.columns)   
print(hb.columns)
print(hbc.columns)


Index(['리뷰', '작성자', '작성자정보', '날짜', '방문회수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자정보', '작성일', '방문횟수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자정보', '날짜', '방문회수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자정보', '작성일', '방문횟수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자정보', '작성일', '방문횟수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자정보', '날짜', '방문회수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자정보', '작성일', '방문횟수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자정보', '날짜', '방문회수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자정보', '날짜', '방문회수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자정보', '날짜', '방문횟수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자정보', '날짜', '방문횟수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자정보', '날짜', '방문횟수', '인증수단', '업소명', '지역'], dtype='object')
Index(['리뷰', '작성자', '작성자

In [4]:
review = pd.concat([gj, npl, dhs, st, scm, cdb, forest, hb, hbc], ignore_index=True)
review.tail()
review.columns

Index(['리뷰', '작성자', '작성자정보', '날짜', '방문횟수', '인증수단', '업소명', '지역'], dtype='object')

In [5]:
# 작성자, 작정자정보, 방문횟수, 인증수단, 작성일, 방문횟수 -> 제거
review = review.drop(columns=['작성자', '작성자정보', '방문횟수', '인증수단', '지역'])
review.iloc[8191:8200]

,리뷰,날짜,업소명
8191,너무너무더웠지만 테마별로넘예쁘게 잘되어있었어용!!!\n생각보다입장료가 비쌋어여ㅠ 경...,2024년 7월 2일 화요일,순천만국가정원
8192,뷰가 너무 이쁘고 좋아요,2024년 9월 28일 토요일,순천만국가정원
8193,"다시 개장한 순천만국가정원.\n개장한지 얼마 안되고, 주말이라 그런지 시민권 발급받...",2024년 4월 6일 토요일,순천만국가정원
8194,NaN,2024년 10월 30일 수요일,순천만국가정원
8195,국가정원 클라스 지립니다\n크기도 엄청커요,2024년 10월 26일 토요일,순천만국가정원
8196,NaN,2024년 11월 23일 토요일,순천만국가정원
8197,너무 멋있어요.. 지방에서 오면 순천사람들과 차이가있는데 그건 쫌~~멀리서 왔는데 ...,2024년 10월 12일 토요일,순천만국가정원
8198,공연보러 갔다왔는데 너무 더워서 힘들었어요ㅜ\n공연은 좋았습니다😂,2024년 8월 3일 토요일,순천만국가정원
8199,good,2024년 10월 29일 화요일,순천만국가정원


In [6]:
# 날짜 형식 변환
# 1. 연, 월, 일 숫자만 추출
review[['연', '월', '일']] = review['날짜'].str.extract(r'(\d{4})년\s*(\d{1,2})월\s*(\d{1,2})일')

# 2. 추출한 숫자로 날짜 문자열 생성 (YYYY-MM-DD)
review['날짜_문자'] = review.apply(
    lambda row: f"{int(row['연']):04d}-{int(row['월']):02d}-{int(row['일']):02d}" 
    if pd.notnull(row['연']) and pd.notnull(row['월']) and pd.notnull(row['일']) 
    else None, axis=1)

# 3. 날짜 문자열을 datetime 타입으로 변환
review['날짜'] = pd.to_datetime(review['날짜_문자'], format='%Y-%m-%d', errors='coerce')

# 4. 임시 컬럼 삭제
review.drop(columns=['연', '월', '일', '날짜_문자'], inplace=True)

# 결과 확인
print(review['날짜'].head())

# 데이터 저장: 계속 이 코드에서 오류가 나서 억지로 데이터를 저장하고 다시 시작
review.to_csv('data/review_processed.csv', index=False, encoding='utf-8-sig')   

0   2025-07-19
1   2025-07-05
2   2025-05-30
3   2025-07-11
4   2025-06-02
Name: 날짜, dtype: datetime64[ns]


In [7]:
# 데이터 불러오기
r2 = pd.read_csv('data/review_processed.csv', encoding='utf-8-sig ')
len(r2) #14391개
r2.head(20)

nat_rows = r2[r2['날짜'].isna()]
print(nat_rows)

r2.iloc[3191:4000]
len(r2)


        리뷰   날짜            업소명
14390  좋아요  NaN  해비치 호텔&리조트 제주


14391

In [8]:
r2['날짜'].dtype #object
r2['날짜'] = pd.to_datetime(r2['날짜'], errors='coerce')
len(r2) #14391개

14391

In [9]:
# 날짜 제한 (2023.01.01 ~ 2025.07.31)까지로 제한
# 다른 데이터들과 어느정도 맞추기 위해서
r2 = r2[(r2['날짜'] >= '2023-01-01') & (r2['날짜'] <= '2025-07-31')]
r2['업소명'].value_counts()
len(r2) #8499개 (5892개는 날짜 조건과 맞지 않아서 아웃)
r2.iloc[7800:8300]

,리뷰,날짜,업소명
12031,우연히 가게됐는데 넘좋았어요. 미리알았다면 한옥카페에서 차를 사가서 마셨을겁니다. ...,2023-05-27,서울한방진흥센터
12032,이런 특강이 많았으면 좋겠습니다 ❤️,2023-08-19,서울한방진흥센터
12033,"족욕과기계식맛사지,박물관전시그리고 다미가에서 차한잔 이 모두가 만원~힐링 제대로 하...",2023-04-15,서울한방진흥센터
12034,아이와 함께 족욕체험 및 전통의상체험 해보았어요.,2023-07-29,서울한방진흥센터
12035,쪼르르 쪼르르따끈한 물이 나온다.우리는 두명이 한스팟이어서 6000원 대이득!더보기,2023-07-13,서울한방진흥센터
...,...,...,...
12716,NaN,2023-06-04,해비치 호텔&리조트 제주
12717,라운딩하기 최고로 좋은 골프장입니다,2023-04-20,해비치 호텔&리조트 제주
12718,수모 빌렷는디 넘 ㄴ좋아요,2023-05-18,해비치 호텔&리조트 제주
12719,또 오고 싶어요.,2023-05-18,해비치 호텔&리조트 제주


In [10]:
14391 - 8499

5892

In [11]:
# 최종 데이터 저장
r2.to_csv('data/review_final.csv', index=False, encoding='utf-8-sig')